# PPM1D Interactive Explorer

This notebook provides an interactive interface to:
1. Configure and run PPM1D simulations
2. Explore history data (global quantities over time)
3. Visualize profile data with interactive controls
4. Compare different dumps and quantities

In [1]:
# Standard imports
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import os
import sys

# Interactive widgets
import ipywidgets as widgets
from IPython.display import display, clear_output

# Add parent directory to path for imports
sys.path.insert(0, '/home/ppathak/1d-hydro-code-ppm/')

# Import PPM1D
from ppm1d import Flags, run_simulation
from ppm1d.grid.grid1d import Grid1D
from ppm1d.initial_conditions import get_initial_condition

print("PPM1D modules loaded successfully!")

PPM1D modules loaded successfully!


## 1. Configure Simulation

Use the interactive widgets below to configure the simulation parameters.

In [2]:
# Create configuration widgets
style = {'description_width': '150px'}
layout = widgets.Layout(width='400px')

# Grid settings
nx_widget = widgets.IntSlider(
    value=200, min=50, max=2000, step=50,
    description='Grid cells (nx):', style=style, layout=layout
)

# Physics settings
gamma_widget = widgets.FloatSlider(
    value=1.4, min=1.1, max=2.0, step=0.1,
    description='Gamma (γ):', style=style, layout=layout
)

# Simulation settings
problem_widget = widgets.Dropdown(
    options=['sod', 'acoustic'],
    value='sod',
    description='Problem type:', style=style, layout=layout
)

t_final_widget = widgets.FloatSlider(
    value=0.2, min=0.05, max=1.0, step=0.05,
    description='Final time:', style=style, layout=layout
)

# Numerics
cfl_widget = widgets.FloatSlider(
    value=0.9, min=0.1, max=0.95, step=0.05,
    description='CFL number:', style=style, layout=layout
)

# Boundary conditions
bc_options = ['reflective', 'outflow', 'periodic']

bc_left_widget = widgets.Dropdown(
    options=bc_options,
    value='outflow',
    description='Left BC:', style=style, layout=layout
)

bc_right_widget = widgets.Dropdown(
    options=bc_options,
    value='outflow',
    description='Right BC:', style=style, layout=layout
)

# Checkbox to use problem-specific default BCs
use_default_bc_widget = widgets.Checkbox(
    value=True,
    description='Use default BCs for problem type',
    style=style,
    layout=layout
)

# Function to update BC widgets when problem changes
def on_problem_change(change):
    if use_default_bc_widget.value:
        if change.new == 'sod':
            bc_left_widget.value = 'outflow'
            bc_right_widget.value = 'outflow'
        elif change.new == 'acoustic':
            bc_left_widget.value = 'periodic'
            bc_right_widget.value = 'periodic'

def on_use_default_change(change):
    # Enable/disable BC widgets based on checkbox
    bc_left_widget.disabled = change.new
    bc_right_widget.disabled = change.new
    # Update to defaults if checked
    if change.new:
        on_problem_change(type('obj', (object,), {'new': problem_widget.value})())

problem_widget.observe(on_problem_change, names='value')
use_default_bc_widget.observe(on_use_default_change, names='value')

# Initialize BC widget state
bc_left_widget.disabled = use_default_bc_widget.value
bc_right_widget.disabled = use_default_bc_widget.value

# Output settings
dump_freq_widget = widgets.FloatSlider(
    value=0.01, min=0.001, max=0.1, step=0.005,
    description='Dump frequency:', style=style, layout=layout
)

output_dir_widget = widgets.Text(
    value='output_notebook',
    description='Output directory:', style=style, layout=layout
)

# Display configuration panel
config_box = widgets.VBox([
    widgets.HTML('<h3>Grid & Physics</h3>'),
    nx_widget, gamma_widget,
    widgets.HTML('<h3>Simulation</h3>'),
    problem_widget, t_final_widget,
    widgets.HTML('<h3>Boundary Conditions</h3>'),
    use_default_bc_widget,
    widgets.HBox([bc_left_widget, bc_right_widget]),
    widgets.HTML('<h3>Numerics</h3>'),
    cfl_widget,
    widgets.HTML('<h3>Output</h3>'),
    dump_freq_widget, output_dir_widget,
])

display(config_box)

## 2. Run Simulation

Click the button below to run the simulation with the configured parameters.

In [3]:
# Global variable to store simulation state
sim_output_dir = None

# Output area for simulation progress
sim_output = widgets.Output()

def run_sim_callback(button):
    """Run simulation with current widget settings."""
    global sim_output_dir
    
    with sim_output:
        clear_output(wait=True)
        
        # Create flags from widget values
        flags = Flags()
        flags.grid.nx = nx_widget.value
        flags.physics.gamma = gamma_widget.value
        flags.simulation.problem = problem_widget.value
        flags.simulation.t_final = t_final_widget.value
        flags.numerics.cfl = cfl_widget.value
        flags.output.soundcrossings_per_dump = dump_freq_widget.value
        flags.output.output_dir = output_dir_widget.value
        flags.output.save_plots = True  # Save PNG plots to disk
        
        # Set boundary conditions
        if use_default_bc_widget.value:
            # Use default BCs for the problem type
            flags.simulation.use_default_bcs = True
            print(f"Using default BCs for '{flags.simulation.problem}' problem")
        else:
            # Use custom BCs from widgets
            flags.simulation.use_default_bcs = False
            flags.grid.bc_left = bc_left_widget.value
            flags.grid.bc_right = bc_right_widget.value
            print(f"Using custom BCs: left={flags.grid.bc_left}, right={flags.grid.bc_right}")
        
        # Store output directory for later use
        sim_output_dir = flags.output.output_dir
        
        print("Starting simulation...\n")
        
        # Run simulation
        final_state = run_simulation(flags)
        
        print(f"\nBoundary conditions used: left={flags.grid.bc_left}, right={flags.grid.bc_right}")
        print("\n" + "="*50)
        print("Simulation complete! You can now explore the results below.")

# Create run button
run_button = widgets.Button(
    description='Run Simulation',
    button_style='success',
    icon='play',
    layout=widgets.Layout(width='200px', height='40px')
)
run_button.on_click(run_sim_callback)

display(run_button)
display(sim_output)

Button(button_style='success', description='Run Simulation', icon='play', layout=Layout(height='40px', width='…

Output()

---

## 3. Data Loading Utilities

Helper functions to load history and profile data.

In [4]:
def load_history(output_dir):
    """
    Load history data from history.data file.
    
    Returns:
        dict: Column name -> numpy array
    """
    history_path = os.path.join(output_dir, 'history', 'history.data')
    
    if not os.path.exists(history_path):
        raise FileNotFoundError(f"History file not found: {history_path}")
    
    # Read column names from header
    with open(history_path, 'r') as f:
        lines = f.readlines()
    
    # Find the column header line (first non-comment line)
    data_start = 0
    col_names = []
    for i, line in enumerate(lines):
        if not line.startswith('#'):
            col_names = line.split()
            data_start = i + 1  # Data starts on the next line
            break
    
    # Load data, skipping all lines up to and including the column header
    data = np.loadtxt(history_path, comments='#', skiprows=data_start)
    
    # Handle single-row case
    if data.ndim == 1:
        data = data.reshape(1, -1)
    
    # Create dictionary
    result = {}
    for i, name in enumerate(col_names):
        result[name] = data[:, i]
    
    return result


def load_profile(output_dir, model_number):
    """
    Load profile data from profile_NNNN.data file.
    
    Returns:
        dict: Column name -> numpy array
        dict: Metadata (model_number, time, step, etc.)
    """
    profile_path = os.path.join(output_dir, 'profiles', f'profile_{model_number:04d}.data')
    
    if not os.path.exists(profile_path):
        raise FileNotFoundError(f"Profile file not found: {profile_path}")
    
    # Read file
    with open(profile_path, 'r') as f:
        lines = f.readlines()
    
    # Parse metadata from header
    metadata = {}
    for line in lines:
        if line.startswith('# ') and '=' in line:
            parts = line[2:].split('=')
            key = parts[0].strip()
            value = parts[1].strip()
            try:
                if '.' in value or 'e' in value:
                    metadata[key] = float(value)
                else:
                    metadata[key] = int(value)
            except:
                metadata[key] = value
    
    # Find column names (first non-comment line)
    data_start = 0
    col_names = []
    for i, line in enumerate(lines):
        if not line.startswith('#'):
            col_names = line.split()
            data_start = i + 1  # Data starts on the next line
            break
    
    # Load data, skipping all lines up to and including the column header
    data = np.loadtxt(profile_path, comments='#', skiprows=data_start)
    
    # Create dictionary
    result = {}
    for i, name in enumerate(col_names):
        result[name] = data[:, i]
    
    return result, metadata


def get_available_dumps(output_dir):
    """
    Get list of available dump numbers.
    
    Returns:
        list: Sorted list of model numbers
    """
    profiles_dir = os.path.join(output_dir, 'profiles')
    if not os.path.exists(profiles_dir):
        return []
    
    dumps = []
    for f in os.listdir(profiles_dir):
        if f.startswith('profile_') and f.endswith('.data'):
            try:
                num = int(f.replace('profile_', '').replace('.data', ''))
                dumps.append(num)
            except:
                pass
    
    return sorted(dumps)


print("Data loading utilities defined.")

Data loading utilities defined.


---

## 4. History Explorer

Visualize global quantities over time from the history file.

In [5]:
# History explorer widgets
history_output = widgets.Output()

# History quantity selection
history_quantities = [
    'total_mass', 'total_energy', 'total_kinetic', 'total_internal',
    'total_momentum', 'max_density', 'min_density', 'max_velocity',
    'max_pressure', 'min_pressure', 'max_mach', 'max_sound_speed'
]

history_y1_widget = widgets.Dropdown(
    options=history_quantities,
    value='total_energy',
    description='Y-axis 1:',
    style=style
)

history_y2_widget = widgets.Dropdown(
    options=['None'] + history_quantities,
    value='total_kinetic',
    description='Y-axis 2:',
    style=style
)

history_log_widget = widgets.Checkbox(
    value=False,
    description='Log scale Y-axis',
    style=style
)

def update_history_plot(y1_col, y2_col, log_scale):
    """Update history plot with selected quantities."""
    with history_output:
        clear_output(wait=True)
        
        if sim_output_dir is None:
            print("Please run a simulation first!")
            return
        
        try:
            history = load_history(sim_output_dir)
        except FileNotFoundError as e:
            print(f"Error: {e}")
            return
        
        time = history['time']
        
        fig, ax1 = plt.subplots(figsize=(12, 6))
        
        # Plot first quantity
        color1 = 'tab:blue'
        ax1.set_xlabel('Time', fontsize=12)
        ax1.set_ylabel(y1_col.replace('_', ' ').title(), color=color1, fontsize=12)
        line1, = ax1.plot(time, history[y1_col], color=color1, linewidth=2, label=y1_col)
        ax1.tick_params(axis='y', labelcolor=color1)
        
        if log_scale:
            ax1.set_yscale('log')
        
        # Plot second quantity if selected
        if y2_col != 'None':
            ax2 = ax1.twinx()
            color2 = 'tab:red'
            ax2.set_ylabel(y2_col.replace('_', ' ').title(), color=color2, fontsize=12)
            line2, = ax2.plot(time, history[y2_col], color=color2, linewidth=2, linestyle='--', label=y2_col)
            ax2.tick_params(axis='y', labelcolor=color2)
            if log_scale:
                ax2.set_yscale('log')
            
            # Combined legend
            lines = [line1, line2]
            labels = [y1_col.replace('_', ' '), y2_col.replace('_', ' ')]
            ax1.legend(lines, labels, loc='upper right')
        else:
            ax1.legend(loc='upper right')
        
        ax1.set_title('History: Global Quantities vs Time', fontsize=14)
        ax1.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

# Create interactive widget
history_interactive = widgets.interactive(
    update_history_plot,
    y1_col=history_y1_widget,
    y2_col=history_y2_widget,
    log_scale=history_log_widget
)

display(widgets.HTML('<h3>History Explorer</h3>'))
display(widgets.HBox([history_y1_widget, history_y2_widget, history_log_widget]))
display(history_output)

# Initial plot
update_history_plot(history_y1_widget.value, history_y2_widget.value, history_log_widget.value)

# Connect widgets
history_y1_widget.observe(lambda change: update_history_plot(change.new, history_y2_widget.value, history_log_widget.value), names='value')
history_y2_widget.observe(lambda change: update_history_plot(history_y1_widget.value, change.new, history_log_widget.value), names='value')
history_log_widget.observe(lambda change: update_history_plot(history_y1_widget.value, history_y2_widget.value, change.new), names='value')

HTML(value='<h3>History Explorer</h3>')

Output()

---

## 5. Profile Explorer

Visualize spatial profiles at different times.

In [6]:
# Profile explorer widgets
profile_output = widgets.Output()

# Profile quantities
profile_quantities = [
    'density', 'velocity', 'pressure', 'total_energy', 'kinetic_energy',
    'internal_energy', 'sound_speed', 'mach_number', 'entropy', 'temperature'
]

# Get initial dump range
def get_dump_range():
    if sim_output_dir is None:
        return 0, 10, 0
    dumps = get_available_dumps(sim_output_dir)
    if len(dumps) == 0:
        return 0, 10, 0
    return min(dumps), max(dumps), min(dumps)

min_dump, max_dump, init_dump = get_dump_range()

dump_slider = widgets.IntSlider(
    value=init_dump, min=min_dump, max=max(max_dump, 1), step=1,
    description='Dump #:',
    style=style,
    layout=widgets.Layout(width='500px'),
    continuous_update=False
)

quantity_dropdown = widgets.Dropdown(
    options=profile_quantities,
    value='density',
    description='Quantity:',
    style=style
)

profile_log_widget = widgets.Checkbox(
    value=False,
    description='Log scale',
    style=style
)

# Play button for animation
play_widget = widgets.Play(
    value=init_dump,
    min=min_dump,
    max=max(max_dump, 1),
    step=1,
    interval=200,
    description="Play",
    disabled=False
)

# Link play to slider
widgets.jslink((play_widget, 'value'), (dump_slider, 'value'))

def update_profile_plot(dump_num, quantity, log_scale):
    """Update profile plot."""
    with profile_output:
        clear_output(wait=True)
        
        if sim_output_dir is None:
            print("Please run a simulation first!")
            return
        
        try:
            profile, metadata = load_profile(sim_output_dir, dump_num)
        except FileNotFoundError as e:
            print(f"Error: {e}")
            return
        
        x = profile['x']
        y = profile[quantity]
        
        fig, ax = plt.subplots(figsize=(12, 6))
        
        ax.plot(x, y, 'b-', linewidth=2)
        ax.fill_between(x, y, alpha=0.3)
        
        ax.set_xlabel('Position (x)', fontsize=12)
        ax.set_ylabel(quantity.replace('_', ' ').title(), fontsize=12)
        ax.set_title(f'{quantity.replace("_", " ").title()} at t = {metadata["time"]:.4f} (Dump {dump_num})', fontsize=14)
        ax.grid(True, alpha=0.3)
        
        if log_scale and np.all(y > 0):
            ax.set_yscale('log')
        
        # Add info box
        info_text = f"Step: {metadata.get('step', 'N/A')}\n"
        info_text += f"Time: {metadata['time']:.6f}\n"
        info_text += f"Min: {np.min(y):.4e}\n"
        info_text += f"Max: {np.max(y):.4e}"
        
        props = dict(boxstyle='round', facecolor='wheat', alpha=0.8)
        ax.text(0.02, 0.98, info_text, transform=ax.transAxes, fontsize=10,
                verticalalignment='top', bbox=props, family='monospace')
        
        plt.tight_layout()
        plt.show()

def refresh_dump_range(button):
    """Refresh the dump slider range after simulation."""
    min_dump, max_dump, _ = get_dump_range()
    dump_slider.min = min_dump
    dump_slider.max = max_dump
    play_widget.min = min_dump
    play_widget.max = max_dump
    dump_slider.value = min_dump
    update_profile_plot(dump_slider.value, quantity_dropdown.value, profile_log_widget.value)

refresh_button = widgets.Button(
    description='Refresh',
    button_style='info',
    icon='refresh'
)
refresh_button.on_click(refresh_dump_range)

display(widgets.HTML('<h3>Profile Explorer</h3>'))
display(widgets.HBox([play_widget, dump_slider, refresh_button]))
display(widgets.HBox([quantity_dropdown, profile_log_widget]))
display(profile_output)

# Connect widgets
dump_slider.observe(lambda change: update_profile_plot(change.new, quantity_dropdown.value, profile_log_widget.value), names='value')
quantity_dropdown.observe(lambda change: update_profile_plot(dump_slider.value, change.new, profile_log_widget.value), names='value')
profile_log_widget.observe(lambda change: update_profile_plot(dump_slider.value, quantity_dropdown.value, change.new), names='value')

HTML(value='<h3>Profile Explorer</h3>')

Output()

---

## 6. Multi-Profile Comparison

Compare profiles at different times on the same plot.

In [7]:
# Multi-profile comparison
multi_output = widgets.Output()

multi_quantity_widget = widgets.Dropdown(
    options=profile_quantities,
    value='density',
    description='Quantity:',
    style=style
)

# Select multiple dumps
multi_dumps_widget = widgets.Text(
    value='0, 10, 20, 30, 40',
    description='Dump numbers:',
    style=style,
    layout=widgets.Layout(width='400px'),
    placeholder='e.g., 0, 10, 20, 30'
)

def plot_multi_profiles(button):
    """Plot multiple profiles on same axes."""
    with multi_output:
        clear_output(wait=True)
        
        if sim_output_dir is None:
            print("Please run a simulation first!")
            return
        
        # Parse dump numbers
        try:
            dump_nums = [int(x.strip()) for x in multi_dumps_widget.value.split(',')]
        except:
            print("Invalid dump numbers. Use comma-separated integers.")
            return
        
        quantity = multi_quantity_widget.value
        
        fig, ax = plt.subplots(figsize=(12, 7))
        
        cmap = cm.viridis
        colors = [cmap(i / max(len(dump_nums)-1, 1)) for i in range(len(dump_nums))]
        
        for i, dump_num in enumerate(dump_nums):
            try:
                profile, metadata = load_profile(sim_output_dir, dump_num)
                x = profile['x']
                y = profile[quantity]
                ax.plot(x, y, color=colors[i], linewidth=2, 
                        label=f't = {metadata["time"]:.4f}')
            except FileNotFoundError:
                print(f"Warning: Dump {dump_num} not found, skipping.")
        
        ax.set_xlabel('Position (x)', fontsize=12)
        ax.set_ylabel(quantity.replace('_', ' ').title(), fontsize=12)
        ax.set_title(f'{quantity.replace("_", " ").title()} Evolution', fontsize=14)
        ax.legend(loc='best', fontsize=10)
        ax.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

multi_plot_button = widgets.Button(
    description='Plot Comparison',
    button_style='primary',
    icon='chart-line'
)
multi_plot_button.on_click(plot_multi_profiles)

display(widgets.HTML('<h3>Multi-Profile Comparison</h3>'))
display(widgets.HBox([multi_quantity_widget, multi_dumps_widget, multi_plot_button]))
display(multi_output)

HTML(value='<h3>Multi-Profile Comparison</h3>')

Output()

---

## 7. Space-Time Diagram

Visualize the evolution of a quantity as a 2D color map (x vs t).

In [8]:
# Space-time diagram
spacetime_output = widgets.Output()

spacetime_quantity_widget = widgets.Dropdown(
    options=profile_quantities,
    value='density',
    description='Quantity:',
    style=style
)

spacetime_cmap_widget = widgets.Dropdown(
    options=['viridis', 'plasma', 'inferno', 'magma', 'cividis', 'coolwarm', 'RdBu_r', 'seismic'],
    value='viridis',
    description='Colormap:',
    style=style
)

def plot_spacetime(button):
    """Create space-time diagram."""
    with spacetime_output:
        clear_output(wait=True)
        
        if sim_output_dir is None:
            print("Please run a simulation first!")
            return
        
        quantity = spacetime_quantity_widget.value
        
        # Load all available dumps
        dumps = get_available_dumps(sim_output_dir)
        if len(dumps) == 0:
            print("No profile files found!")
            return
        
        print(f"Loading {len(dumps)} profiles...")
        
        # Collect data
        data_list = []
        times = []
        x_grid = None
        
        for dump_num in dumps:
            try:
                profile, metadata = load_profile(sim_output_dir, dump_num)
                data_list.append(profile[quantity])
                times.append(metadata['time'])
                if x_grid is None:
                    x_grid = profile['x']
            except:
                pass
        
        if len(data_list) == 0:
            print("No valid profile data found!")
            return
        
        # Create 2D array (time x space)
        data_2d = np.array(data_list)
        times = np.array(times)
        
        # Plot
        fig, ax = plt.subplots(figsize=(12, 8))
        
        im = ax.pcolormesh(x_grid, times, data_2d, 
                           cmap=spacetime_cmap_widget.value, shading='auto')
        
        cbar = plt.colorbar(im, ax=ax, label=quantity.replace('_', ' ').title())
        
        ax.set_xlabel('Position (x)', fontsize=12)
        ax.set_ylabel('Time', fontsize=12)
        ax.set_title(f'Space-Time Diagram: {quantity.replace("_", " ").title()}', fontsize=14)
        
        plt.tight_layout()
        plt.show()

spacetime_button = widgets.Button(
    description='Generate Diagram',
    button_style='warning',
    icon='th'
)
spacetime_button.on_click(plot_spacetime)

display(widgets.HTML('<h3>Space-Time Diagram</h3>'))
display(widgets.HBox([spacetime_quantity_widget, spacetime_cmap_widget, spacetime_button]))
display(spacetime_output)

HTML(value='<h3>Space-Time Diagram</h3>')

Output()

---

## 8. Conservation Check

Verify conservation of mass, momentum, and energy.

In [9]:
# Conservation check
conservation_output = widgets.Output()

def plot_conservation(button):
    """Plot conservation quantities."""
    with conservation_output:
        clear_output(wait=True)
        
        if sim_output_dir is None:
            print("Please run a simulation first!")
            return
        
        try:
            history = load_history(sim_output_dir)
        except FileNotFoundError as e:
            print(f"Error: {e}")
            return
        
        time = history['time']
        
        # Compute relative changes
        mass_0 = history['total_mass'][0]
        energy_0 = history['total_energy'][0]
        
        mass_rel = (history['total_mass'] - mass_0) / mass_0
        energy_rel = (history['total_energy'] - energy_0) / energy_0
        momentum = history['total_momentum']  # Should be ~0 for symmetric problems
        
        fig, axes = plt.subplots(3, 1, figsize=(12, 10), sharex=True)
        
        # Mass conservation
        axes[0].plot(time, mass_rel * 100, 'b-', linewidth=2)
        axes[0].axhline(y=0, color='k', linestyle='--', alpha=0.5)
        axes[0].set_ylabel('Mass Change (%)', fontsize=12)
        axes[0].set_title('Conservation Check', fontsize=14)
        axes[0].grid(True, alpha=0.3)
        axes[0].text(0.02, 0.98, f'Max deviation: {np.max(np.abs(mass_rel))*100:.2e}%',
                     transform=axes[0].transAxes, fontsize=10, va='top',
                     bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
        
        # Energy conservation
        axes[1].plot(time, energy_rel * 100, 'r-', linewidth=2)
        axes[1].axhline(y=0, color='k', linestyle='--', alpha=0.5)
        axes[1].set_ylabel('Energy Change (%)', fontsize=12)
        axes[1].grid(True, alpha=0.3)
        axes[1].text(0.02, 0.98, f'Max deviation: {np.max(np.abs(energy_rel))*100:.2e}%',
                     transform=axes[1].transAxes, fontsize=10, va='top',
                     bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
        
        # Momentum
        axes[2].plot(time, momentum, 'g-', linewidth=2)
        axes[2].axhline(y=0, color='k', linestyle='--', alpha=0.5)
        axes[2].set_xlabel('Time', fontsize=12)
        axes[2].set_ylabel('Total Momentum', fontsize=12)
        axes[2].grid(True, alpha=0.3)
        axes[2].text(0.02, 0.98, f'Max |momentum|: {np.max(np.abs(momentum)):.2e}',
                     transform=axes[2].transAxes, fontsize=10, va='top',
                     bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
        
        plt.tight_layout()
        plt.show()

conservation_button = widgets.Button(
    description='Check Conservation',
    button_style='success',
    icon='check'
)
conservation_button.on_click(plot_conservation)

display(widgets.HTML('<h3>Conservation Check</h3>'))
display(conservation_button)
display(conservation_output)

HTML(value='<h3>Conservation Check</h3>')

Button(button_style='success', description='Check Conservation', icon='check', style=ButtonStyle())

Output()

---

## 9. Three-Panel Dashboard

Classic Sod shock tube visualization with density, velocity, and pressure.

In [10]:
# Three-panel dashboard
dashboard_output = widgets.Output()

dashboard_dump_slider = widgets.IntSlider(
    value=0, min=0, max=100, step=1,
    description='Dump #:',
    style=style,
    layout=widgets.Layout(width='600px'),
    continuous_update=False
)

dashboard_play = widgets.Play(
    value=0, min=0, max=100, step=1,
    interval=150, description="Animate"
)

widgets.jslink((dashboard_play, 'value'), (dashboard_dump_slider, 'value'))

def update_dashboard(dump_num):
    """Update three-panel dashboard."""
    with dashboard_output:
        clear_output(wait=True)
        
        if sim_output_dir is None:
            print("Please run a simulation first!")
            return
        
        try:
            profile, metadata = load_profile(sim_output_dir, dump_num)
        except FileNotFoundError as e:
            print(f"Error: {e}")
            return
        
        x = profile['x']
        
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        
        # Density
        axes[0].plot(x, profile['density'], 'b-', linewidth=2)
        axes[0].fill_between(x, profile['density'], alpha=0.3)
        axes[0].set_xlabel('Position')
        axes[0].set_ylabel('Density')
        axes[0].set_title('Density')
        axes[0].grid(True, alpha=0.3)
        
        # Velocity
        axes[1].plot(x, profile['velocity'], 'r-', linewidth=2)
        axes[1].fill_between(x, profile['velocity'], alpha=0.3, color='red')
        axes[1].set_xlabel('Position')
        axes[1].set_ylabel('Velocity')
        axes[1].set_title('Velocity')
        axes[1].grid(True, alpha=0.3)
        
        # Pressure
        axes[2].plot(x, profile['pressure'], 'g-', linewidth=2)
        axes[2].fill_between(x, profile['pressure'], alpha=0.3, color='green')
        axes[2].set_xlabel('Position')
        axes[2].set_ylabel('Pressure')
        axes[2].set_title('Pressure')
        axes[2].grid(True, alpha=0.3)
        
        fig.suptitle(f't = {metadata["time"]:.4f} (Dump {dump_num}, Step {metadata.get("step", "N/A")})', 
                     fontsize=14, y=1.02)
        
        plt.tight_layout()
        plt.show()

def refresh_dashboard(button):
    """Refresh dashboard dump range."""
    min_dump, max_dump, _ = get_dump_range()
    dashboard_dump_slider.min = min_dump
    dashboard_dump_slider.max = max_dump
    dashboard_play.min = min_dump
    dashboard_play.max = max_dump
    dashboard_dump_slider.value = min_dump
    update_dashboard(min_dump)

dashboard_refresh = widgets.Button(
    description='Refresh',
    button_style='info',
    icon='refresh'
)
dashboard_refresh.on_click(refresh_dashboard)

display(widgets.HTML('<h3>Three-Panel Dashboard (Density, Velocity, Pressure)</h3>'))
display(widgets.HBox([dashboard_play, dashboard_dump_slider, dashboard_refresh]))
display(dashboard_output)

# Connect
dashboard_dump_slider.observe(lambda change: update_dashboard(change.new), names='value')

HTML(value='<h3>Three-Panel Dashboard (Density, Velocity, Pressure)</h3>')

Output()

---

## 10. Quick Summary

Display a summary of the simulation results.

In [24]:
def print_summary():
    """Print simulation summary."""
    if sim_output_dir is None:
        print("No simulation has been run yet.")
        return
    
    try:
        history = load_history(sim_output_dir)
        dumps = get_available_dumps(sim_output_dir)
    except:
        print("Could not load simulation data.")
        return
    
    print("=" * 60)
    print("SIMULATION SUMMARY")
    print("=" * 60)
    print(f"\nOutput directory: {sim_output_dir}")
    print(f"Number of dumps: {len(dumps)}")
    print(f"Time range: {history['time'][0]:.4f} -> {history['time'][-1]:.4f}")
    print(f"Total steps: {int(history['step'][-1])}")
    print(f"\nConservation:")
    print(f"  Mass change: {(history['total_mass'][-1]/history['total_mass'][0] - 1)*100:.2e}%")
    print(f"  Energy change: {(history['total_energy'][-1]/history['total_energy'][0] - 1)*100:.2e}%")
    print(f"\nFinal state:")
    print(f"  Max density: {history['max_density'][-1]:.4f}")
    print(f"  Max velocity: {history['max_velocity'][-1]:.4f}")
    print(f"  Max pressure: {history['max_pressure'][-1]:.4f}")
    print(f"  Max Mach: {history['max_mach'][-1]:.4f}")
    print("=" * 60)

print_summary()

SIMULATION SUMMARY

Output directory: acoustic-output
Number of dumps: 110
Time range: 0.0000 -> 1.0000
Total steps: 265

Conservation:
  Mass change: 0.00e+00%
  Energy change: 9.55e-13%

Final state:
  Max density: 1.0050
  Max velocity: 0.0059
  Max pressure: 1.0070
  Max Mach: 0.0050
